In [1]:
%matplotlib widget

# Calculate the Luminosity

In [2]:
import numpy as np
import xobjects as xo
import xtrack as xt
import xfields as xf
import xpart as xp

import pandas as pd

### Get the line

In [3]:
# Load the lattice
#env = xt.load_madx_lattice('ffs_clic_7TeV.madx')

# Choose the line that we want to use
#line_b1 = env['ffs']
#line_b2 = env['ffs']

# Create the context
test_context = xo.ContextCpu()

#line_b1.build_tracker(_context=test_context)
#line_b2.build_tracker(_context=test_context)

### Read in a text file with the beam parameters

In [4]:
df = pd.read_csv("Barklow_Wakefield_15TeV.txt", delimiter=',')
print(df)

    Variable         Value   Units
0     Energy  7.500000e+12      eV
1  Intensity  5.000000e+09    none
2      BetaX  1.500000e-04       m
3      BetaY  1.500000e-04       m
4       em_X  1.000000e-07   m rad
5       em_Y  1.000000e-07   m rad
6     SigmaZ  5.000000e-06       m
7      Delta  1.000000e-02    none
8        Phi  0.000000e+00     rad
9        Rep  7.730000e+03      Hz


In [5]:
df[['Variable']]

,Variable
0,Energy
1,Intensity
2,BetaX
3,BetaY
4,em_X
5,em_Y
6,SigmaZ
7,Delta
8,Phi
9,Rep


In [6]:
#df.iloc[0,0]  #Access value at this particular index 

#for index, row in df.iterrows():
   # print(f"Index: {index}, Col1: {row['Variable']}")

#############################################################
# Initialize the beam variables for the lumi calculation
#############################################################

_energy = 0.0
_intensity = 0.0
_beta_x = 0.0
_beta_y = 0.0
_emit_x = 0.0
_emit_y = 0.0
_sigma_z = 0.0
_delta = 0.0
_phi = 0.0
_rep = 0.0

# Get the size of the array for the loop
n_rows = len(df)

# Loop over the number of rows
for k in range(0,n_rows):
    if (df.iloc[k,0] == 'Energy'):
        _energy = df.iloc[k,1]
    if (df.iloc[k,0] == 'Intensity'):
        _intensity = df.iloc[k,1]
    if (df.iloc[k,0] == 'BetaX'):
        _beta_x = df.iloc[k,1]
    if (df.iloc[k,0] == 'BetaY'):
        _beta_y = df.iloc[k,1]
    if (df.iloc[k,0] == 'em_X'):
        _emit_x = df.iloc[k,1]
    if (df.iloc[k,0] == 'em_Y'):
        _emit_y = df.iloc[k,1]
    if (df.iloc[k,0] == 'SigmaZ'):
        _sigma_z = df.iloc[k,1]
    if (df.iloc[k,0] == 'Delta'):
        _delta = df.iloc[k,1]
    if (df.iloc[k,0] == 'Phi'):
        _delta = df.iloc[k,1]
    if (df.iloc[k,0] == 'Rep'):
        _rep = df.iloc[k,1]
    

#print("Energy = %f"%(_energy))

In [7]:
#ex = 0.1e-06
#bx = 0.15e-03
#np.sqrt(ex*bx/1.47e7)

### Define the beams

In [8]:
### Global beam parameters
energy = float(_energy) # in eV
n_part_b1 = int(1e6)  # number of macroparticles in e- beam
n_part_b2 = int(1e6)  # number of macroparticles in e+ beam
bunch_intensity = float(_intensity)
beta_x = float(_beta_x)
beta_y = float(_beta_y)
gamma = float(_energy / xt.ELECTRON_MASS_EV)
physemit_x = float(_emit_x / gamma)
physemit_y = float(_emit_y / gamma)
sigma_x = float(np.sqrt(beta_x*physemit_x))
sigma_px = float(np.sqrt(physemit_x/beta_x))
sigma_y = float(np.sqrt(beta_y*physemit_y))
sigma_py = float(np.sqrt(physemit_y/beta_y))
sigma_z_tot = float(_sigma_z)   # bunch length
sigma_delta_tot = float(_delta)  # Energy spread of beam
phi = float(_phi) # crossing angle

#print("physemit_y = %.15f"%(physemit_y))


### Electron Beam
particles_b1_pic = xp.Particles(
    _context = test_context,
    q0       = -1,
    p0c      = energy,
    mass0    = xt.ELECTRON_MASS_EV,
    x        = sigma_x            * np.random.randn(n_part_b1),
    y        = sigma_y            * np.random.randn(n_part_b1),
    zeta     = sigma_z_tot        * np.random.randn(n_part_b1),
    px       = sigma_px           * np.random.randn(n_part_b1),
    py       = sigma_py           * np.random.randn(n_part_b1),
    delta    = sigma_delta_tot    * np.random.randn(n_part_b1),
    weight   = bunch_intensity/n_part_b1
)

### Positron Beam
particles_b2_pic = xp.Particles(
    _context = test_context,
    q0       = 1,
    p0c      = energy,
    mass0    = xt.ELECTRON_MASS_EV,
    x        = sigma_x            * np.random.randn(n_part_b2),
    y        = sigma_y            * np.random.randn(n_part_b2),
    zeta     = sigma_z_tot        * np.random.randn(n_part_b2),
    px       = sigma_px           * np.random.randn(n_part_b2),
    py       = sigma_py           * np.random.randn(n_part_b2),
    delta    = sigma_delta_tot    * np.random.randn(n_part_b2),
    weight   = bunch_intensity/n_part_b2
)

print("sigma_x = %.2e"%(sigma_x))
print("sigma_y = %.2e"%(sigma_y))
print("gamma = %.2e"%(gamma))

sigma_x = 1.01e-09
sigma_y = 1.01e-09
gamma = 1.47e+07


In [9]:
####################
# specify PIC grid #
####################
    
nx = 64
ny = 64
nz = 64
x_lim = 6
y_lim = 6
z_lim = 6
    
x_lim_grid = x_lim * sigma_x
y_lim_grid = y_lim * sigma_y
z_lim_grid = z_lim * sigma_z_tot
    
dx = 2*x_lim_grid/(nx)
dy = 2*y_lim_grid/(ny)
dz = 2*z_lim_grid/(nz)
    
bbpic_ip1_b1 = xf.BeamBeamPIC3D(
    _context=test_context,
    phi=phi, alpha=0,
    x_range=(-x_lim_grid, x_lim_grid), dx=dx,
    y_range=(-y_lim_grid, y_lim_grid), dy=dy,
    z_range=(-z_lim_grid, z_lim_grid), dz=dz,
    flag_luminosity=1
    )
    
bbpic_ip1_b2 = xf.BeamBeamPIC3D(
    _context=test_context,
    phi=-phi, alpha=0,
    x_range=(-x_lim_grid, x_lim_grid), dx=dx,
    y_range=(-y_lim_grid, y_lim_grid), dy=dy,
    z_range=(-z_lim_grid, z_lim_grid), dz=dz,
    flag_luminosity=1, 
    )

#print("Sum = %.30f"%(np.sum(bbpic_ip1_b1.fieldmap_self.rho)))
#print("Sum = %.30f"%(np.sum(bbpic_ip1_b2.fieldmap_self.rho)))

#print("x_lim_grid = %.15f"%(x_lim_grid))
#print("y_lim_grid = %.15f"%(y_lim_grid))
#print("z_lim_grid = %.15f"%(z_lim_grid))

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [10]:
############################
# Set up communicator
############################

pipeline_manager = xt.PipelineManager()
pipeline_manager.add_particles('p_b1', rank=0)
pipeline_manager.add_particles('p_b2', rank=0)

pipeline_manager.add_element('IP')
bbpic_ip1_b1.name = 'IP'
bbpic_ip1_b2.name = 'IP'
bbpic_ip1_b1.partner_name = 'p_b2'
bbpic_ip1_b2.partner_name = 'p_b1'
particles_b1_pic.init_pipeline('p_b1')
particles_b2_pic.init_pipeline('p_b2')
bbpic_ip1_b1.pipeline_manager = pipeline_manager
bbpic_ip1_b2.pipeline_manager = pipeline_manager

########################################
# Set5 up xtrack line
########################################
line_b1 = xt.Line(elements=[bbpic_ip1_b1])
line_b2 = xt.Line(elements=[bbpic_ip1_b2])

line_b1.build_tracker(_context=test_context)
line_b2.build_tracker(_context=test_context)

### Set up the multitracker
multitracker = xt.PipelineMultiTracker(
    branches=[xt.PipelineBranch(line=line_b1, particles=particles_b1_pic),
              xt.PipelineBranch(line=line_b2, particles=particles_b2_pic)],
    )

In [11]:
##############################
# Configure record table
##############################
record_b1_pic = line_b1.start_internal_logging_for_elements_of_type(
    xf.BeamBeamPIC3D, capacity={"beamstrahlungtable": int(0), "lumitable": int(1)}
)
record_b2_pic = line_b2.start_internal_logging_for_elements_of_type(
    xf.BeamBeamPIC3D, capacity={"beamstrahlungtable": int(0), "lumitable": int(1)}
)

In [12]:
##############################
# Track 1 collision
##############################

multitracker.track(num_turns=1)

line_b1.stop_internal_logging_for_elements_of_type(xf.BeamBeamPIC3D)
line_b2.stop_internal_logging_for_elements_of_type(xf.BeamBeamPIC3D)

record_b1_pic.move(_context=xo.context_default)
record_b2_pic.move(_context=xo.context_default)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


/sdf/group/facet/kdownham/miniforge3/xfields/xfields/beam_elements/beambeam3dpic.py:427: RuntimeWarning: divide by zero encountered in scalar divide
  scale1 = 1.0 / (sum1 * dx * dy * dz)
/sdf/group/facet/kdownham/miniforge3/xfields/xfields/beam_elements/beambeam3dpic.py:430: RuntimeWarning: invalid value encountered in multiply
  h1_scaled = lumigrid_my_beam * scale1


In [13]:
record_b1_pic.lumitable.luminosity[0]

np.float64(1.0967945887042164e+37)

In [14]:
record_b2_pic.lumitable.luminosity[0]

np.float64(1.096794588704216e+37)

In [15]:
#f_rep = 7.73e3 # Number of repetitions (in Hertz)
conversion = 1e-4 #convert to cm^-2
lumi_1coll = float(record_b1_pic.lumitable.luminosity[0])

#print("Luminosity = %.2e cm^-2 s^-1"%(conversion*f_rep*lumi_1coll))

lumi_bc = conversion * lumi_1coll

print("Luminosity per bunch crossing = %.2e [cm^-2]"%(lumi_bc))

Luminosity per bunch crossing = 1.10e+33 [cm^-2]


In [16]:
piwi = sigma_z_tot / sigma_x * phi
lumi_ip = conversion*bunch_intensity**2 / (4*np.pi*sigma_x*np.sqrt(1 + piwi**2)*sigma_y)

In [17]:
print("Luminosity per bunch crossing = %.2e [cm^-2]"%(lumi_ip))

Luminosity per bunch crossing = 1.95e+32 [cm^-2]


In [18]:
f_rep = _rep # repetition rate [Hz]
print("Luminosity (PIC): %.2e [cm-2 s-1]"%(f_rep*lumi_bc))
print("Luminosity (Formula): %.2e [cm-2 s-1]"%(f_rep*lumi_ip))

Luminosity (PIC): 8.48e+36 [cm-2 s-1]
Luminosity (Formula): 1.50e+36 [cm-2 s-1]
